In [1]:
# first, import the circuit benchmarks

# second, run transpilation on benchmarks for many iterations and save results
# want to show how much variance there is in the results
# NOTE important, use the same placement and routing before passed to transpiler so has a fair comparison

# third, use longest path to convert to fidelities
# we want to calculate expected fidelity improvement factor
# is fidelity improvement factor the same as the ratio between lengths (?)
# 

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR) # turn off for now

#  CONSTANTS
# repeat 10 times, only keep best
data_repitition = range(10)
average_repitition = range(3)

# 1Q:2Q gate duraton ratio
duration_1q = .25

# number of qubits
n_qubits = 16

In [3]:
from slam.utils.transpiler_pass.speed_limit_pass import \
    pass_manager_basic, \
    pass_manager_optimized_sqiswap
    # pass_manager_slam

""" 
Basic, uses alibaba rules
Optimized, use parallel gate optimized CPhase-family and SWAP ruless
"""
speed_method = 'linear' # just using iswap-family gates the SLF makes no difference for now because they all scale linearly to each other
pm_basic = pass_manager_basic(gate='sqiswap', duration_1q=duration_1q)
pm_optimized = pass_manager_optimized_sqiswap(duration_1q=duration_1q, speed_method=speed_method)

In [4]:
import numpy as np
def fidelity(qc_duration, n_qubits=1.0):
    """simple model for fidelity on a single msmt outcome"""
    # iswap:=1 in previous units
    # iswap time is 100 ns
    iswap_time = 1e-7
    # coherence time is 100 us
    t1 = 1e-4
    return (np.exp(-qc_duration*iswap_time/t1))**n_qubits

def infidelity_factor(qc_time1, qc_time2, n_qubits=1):
    """Note, for these chosen values, given a fidelity threshold of 0.5 fidelity, duration <= 69.3"""
    if1 = 1 - fidelity(qc_time1, n_qubits=n_qubits)
    if2 = 1 - fidelity(qc_time2, n_qubits=n_qubits)
    # percent change
    return 100*(if1-if2)/if1

def fidelity_factor(qc_time1, qc_time2, n_qubits=1):
    """Note, for these chosen values, given a fidelity threshold of 0.5 fidelity, duration <= 69.3"""
    f1 = fidelity(qc_time1, n_qubits=n_qubits)
    f2 = fidelity(qc_time2, n_qubits=n_qubits)
    # percent change
    return 100*(f2-f1)/f1

In [6]:
from qiskit import QuantumCircuit
qc = QuantumCircuit(2)
qc.swap(0,1) # or qc.cx(0,1)

output= pm_basic.run(qc)
duration1 = pm_basic.property_set['duration']
print(duration1)
# print(output.draw())
# print(1 - fidelity(duration1, n_qubits=2))
# round to 4 sig figs
print(round(1 - fidelity(duration1, n_qubits=2), 4))
#
output= pm_optimized.run(qc)
duration2 = pm_optimized.property_set['duration']
print(duration2)
# print(output.draw())
# print(1 - fidelity(duration2, n_qubits=2))
# round to 4 sig figs
print(round(1 - fidelity(duration2, n_qubits=2), 4))

pimprov = infidelity_factor(duration1, duration2, n_qubits=2)
# print(pimprov)
print(round(pimprov, 2))


2.5
0.005
2.25
0.0045
9.98


In [13]:
1- fidelity(2.1475, n_qubits=2)
1- fidelity(1.8975, n_qubits=2)
infidelity_factor(2.1475, 1.8975, n_qubits=2)

11.619366034265234

In [12]:
# percent decrease between 2.1475 and 1.8975
11.6414

1.8835855646100117

In [13]:
# unit infidelities over haar
from qiskit.quantum_info import random_unitary
avg_dur1 = 0
avg_dur2 = 0
N=3000
for i in range(N):
    U = random_unitary(4)
    qc = QuantumCircuit(2)
    qc.unitary(U, [0,1])
    output= pm_basic.run(qc)
    duration1 = pm_basic.property_set['duration']
    avg_dur1 += duration1
    output= pm_optimized.run(qc)
    duration2 = pm_optimized.property_set['duration']
    avg_dur2 += duration2

avg_dur1 /= N
avg_dur2 /= N
print(avg_dur1)
print(round(1 - fidelity(avg_dur1, n_qubits=2), 4))
print(avg_dur2)
print(round(1 - fidelity(avg_dur2, n_qubits=2), 4))
print(infidelity_factor(avg_dur1, avg_dur2, n_qubits=2))

1.9045
0.0038
1.36525
0.0027
28.275872503098906


In [21]:
# first, import the circuit benchmarks
from slam.utils.circuit_suite import benchmark_lambdas

# define the topology
from qiskit import transpile
from qiskit.transpiler import CouplingMap
coupling_map = CouplingMap.from_grid(4,4)
induce_swaps = lambda qc: transpile(qc, coupling_map=coupling_map, optimization_level=3)

# repeat, only keep best
for benchmark_circuit in benchmark_lambdas:

    basic_data = [] #duration
    optimized_data = []

    # averaging loop
    # if benchmark_circuit.__name__ == "QV":
    #     avg_rep = average_repitition
    # else:
    #     avg_rep = range(1)
    # for _ in avg_rep:
    for _ in average_repitition:

        # best, defined by shortest duration
        basic_best = (None, None, None, None)
        optimized_best = (None, None, None, None)

        for _ in data_repitition:
            qc = benchmark_circuit(n_qubits) # instantiate of size 16

            # before inducing, turn off logging for readability
            logger.setLevel(logging.ERROR)
            # NOTE non-determinsitc, hence the averaging
            qc = induce_swaps(qc) # transpile to induce swaps

            # second, run the benchmark
            qc_basic = pm_basic.run(qc)
            duration = pm_basic.property_set['duration']
            gate_counts = pm_basic.property_set['gate_counts']
            crit_counts = pm_basic.property_set['longest_path_counts']

            # check if best
            if basic_best[0] is None or duration < basic_best[0]:
                basic_best = (duration, gate_counts, crit_counts, qc_basic)
            
            # repeat on optimized
            qc_optimized = pm_optimized.run(qc)
            duration = pm_optimized.property_set['duration']
            gate_counts = pm_optimized.property_set['gate_counts']
            crit_counts = pm_optimized.property_set['longest_path_counts']

            # check if best
            if optimized_best[0] is None or duration < optimized_best[0]:
                optimized_best = (duration, gate_counts, crit_counts, qc_optimized)
                
        # save results to average
        assert basic_best[0] >= optimized_best[0]
        basic_data.append(basic_best[0])
        optimized_data.append(optimized_best[0])

    # compute averages with std dev
    basic_average = np.mean(basic_data)
    optimized_average = np.mean(optimized_data)
    basic_err = np.std(basic_data)
    optimized_err = np.std(optimized_data)

    # print results
    print(benchmark_circuit.__name__)
    # print basic w/err and optimized w/err
    print("basic duration", basic_average, basic_err)
    print("optimized duration", optimized_average, optimized_err)
    print('___')
    print("duration % reduction", 100*(basic_average - optimized_average)/basic_average)
    print("wire fidelity -> optimized fidelity", fidelity(basic_average), fidelity(optimized_average))
    print("wire fidelity % improvement", fidelity_factor(basic_average, optimized_average))
    # print("wire infidelity % reduction", infidelity_factor(basic_average, optimized_average))
    print("total fidelity -> optimized fidelity", fidelity(basic_average, n_qubits=n_qubits), fidelity(optimized_average, n_qubits=n_qubits))
    # print("total infidelity % reduction", infidelity_factor(basic_average, optimized_average, n_qubits=n_qubits))
    print("total fidelity % improvement", fidelity_factor(basic_average, optimized_average, n_qubits=n_qubits))
    print("\n")